In [1]:
#!pip install tensorflow

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters
import pandas as pd
import sklearn

2025-09-01 07:53:20.786234: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 07:53:20.855346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-01 07:53:22.548005: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
# word_index = reuters.get_word_index(path="reuters_word_index.json")
#!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/reuters_word_index.json

import json

# 1. 파일 열기
with open('reuters_word_index.json', 'r', encoding='utf-8') as f:
    # 2. JSON 로드
    word_index = json.load(f)

print('=3')

=3


In [4]:
word_index['the']

1

In [5]:
word_index['it']

13

In [6]:
index_to_word = { index+3 : word for word, index in word_index.items() }
print('=3')

=3


In [7]:
print(index_to_word[4])
print(index_to_word[16])

the
it


In [8]:
# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token
print('=3')

=3


In [9]:
num_words = [None, 10000, 5000] # 단어 개수별 실험 준비
num_words

[None, 10000, 5000]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
print('=3')

=3


In [11]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

print('=3')

=3


In [13]:
for NUM in num_words:
    if NUM == 10000:
        print('***** 10,000개 단어 *****')
    elif NUM == 5000:
        print('***** 5,000개 단어 *****')
    else:
        print('***** 모든 단어 *****')
    
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=NUM, test_split=0.2) # 데이터 직접 가져옴

    decoded = []
    for i in range(len(x_train)):
        t = ' '.join([index_to_word[index] for index in x_train[i]])
        decoded.append(t)
    
    x_train = decoded

    decoded = []
    for i in range(len(x_test)):
        t = ' '.join([index_to_word[index] for index in x_test[i]])
        decoded.append(t)
    
    x_test = decoded

    dtmvector = CountVectorizer()
    x_train_dtm = dtmvector.fit_transform(x_train)
    print(x_train_dtm.shape)

    tfidf_transformer = TfidfTransformer()
    tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
    print(tfidfv.shape)

    
    print("--- (1) 나이브 베이즈 분류기 ---")
    model = MultinomialNB()
    model.fit(tfidfv, y_train)
    x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
    tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환
    
    predicted = model.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    # model.predict(tfidfv_test[3])

    # ## 5. F1-Score, Confusion Matrix
    # print(classification_report(y_test, model.predict(tfidfv_test), zero_division=0))

    print("--- (2) Complement Naive Bayes Classifier(CNB) ---")
    cb = ComplementNB()
    cb.fit(tfidfv, y_train)
    predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (3) 로지스틱 회귀(Logistic Regression) ---")
    lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
    lr.fit(tfidfv, y_train)
    predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (4) 선형 서포트 벡터 머신(Linear Support Vector Machine) ---")
    lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
    lsvc.fit(tfidfv, y_train)
    predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (5) 결정 트리(Decision Tree) ---")
    tree = DecisionTreeClassifier(max_depth=10, random_state=0)
    tree.fit(tfidfv, y_train)
    predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (6) 랜덤 포레스트(Random Forest) ---")
    forest = RandomForestClassifier(n_estimators=5, random_state=0)
    forest.fit(tfidfv, y_train)
    predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (7) 그래디언트 부스팅 트리(GradientBoostingClassifier) ---")
    grbt = GradientBoostingClassifier(random_state=0) # verbose=3
    grbt.fit(tfidfv, y_train)
    predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

    print("--- (8) 보팅(Voting) ---")
    voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))
    ], voting='soft', n_jobs=-1)
    voting_classifier.fit(tfidfv, y_train)
    predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
    print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교
    print("F1 Score:", f1_score(y_test, predicted, average='macro'))

***** 모든 단어 *****
(8982, 26506)
(8982, 26506)
--- (1) 나이브 베이즈 분류기 ---
정확도: 0.5997328584149599
F1 Score: 0.06767886443611608
--- (2) Complement Naive Bayes Classifier(CNB) ---
정확도: 0.7649154051647373
F1 Score: 0.4639854358945848
--- (3) 로지스틱 회귀(Logistic Regression) ---
정확도: 0.811219946571683
F1 Score: 0.6629074440453098
--- (4) 선형 서포트 벡터 머신(Linear Support Vector Machine) ---


/opt/conda/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


정확도: 0.7929652715939448
F1 Score: 0.6694322208704948
--- (5) 결정 트리(Decision Tree) ---
정확도: 0.6211041852181657
F1 Score: 0.1545005540480078
--- (6) 랜덤 포레스트(Random Forest) ---
정확도: 0.6544968833481746
F1 Score: 0.2794631711818573
--- (7) 그래디언트 부스팅 트리(GradientBoostingClassifier) ---


KeyboardInterrupt: 

# 회고
- 다양한 비교군을 두고 실험하는 것이 의미가 있고
- 잘 모듈화 시켜 놓으면 다른 데이터에 대해서도 해당 모듈을 이용해서 보다 빠르게 성능 비교를 해볼 수 있을것 같음
- 시간이 모자라서 실험을 다 끝내지 못함